# Create an integration object so that we can access the AWS S3 Bucket.





In [38]:
import code_library as cl
from dotenv import dotenv_values

In [30]:
session = cl.snowconnection(cl.connection_parameters)

In [31]:
print(session.sql(
    f"""
  create or replace storage integration s3_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = S3
  ENABLED = TRUE 
  STORAGE_AWS_ROLE_ARN = '{cl.secrets['STORAGE_AWS_ROLE_ARN']}'
  STORAGE_ALLOWED_LOCATIONS = ('{cl.secrets['s3_bucket_for_stg']}')
  COMMENT = 'This is the integration object we are going to use to create the snowflake external stage' ;
    """
    
    ).collect())

[Row(status='Integration S3_INT successfully created.')]


In [33]:
result = session.sql(
    f"""
        DESC integration s3_int;
    """
    )

In [16]:
print(result)
print(type(result))
# the below command is working, commenting it out to keep the secrets a secret :)
# result.show()

<class 'snowflake.snowpark.dataframe.DataFrame'>


In [44]:
print(session.sql(
    f"""
    alter integration s3_int
    set STORAGE_ALLOWED_LOCATIONS = ('s3://inzs3bucketlearn/');
    """
    
    ).collect())



[Row(status='Statement executed successfully.')]


## Now add the storage_aws_iam_user_arn and STORAGE_AWS_EXTERNAL_ID in the trust policy of the aws role which was used while creating the integration object

### below shows the trust policy of the role used while creating the integration object
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Effect": "Allow",
			"Principal": {
				"AWS": "<<add  storage_aws_iam_user_arn from snoflake here>>"
			},
			"Action": "sts:AssumeRole",
			"Condition": {
				"StringEquals": {
					"sts:ExternalId": "<< add STORAGE_AWS_EXTERNAL_ID here>>"
				}
			}
		}
	]
}

In [19]:
# create a file format object
result = session.sql(
    f"""
        CREATE OR REPLACE SCHEMA DEMO_DB.file_formats;
    """
    )

result.show()

---------------------------------------------
|"status"                                   |
---------------------------------------------
|Schema FILE_FORMATS successfully created.  |
---------------------------------------------



In [20]:
# create a file format object
result = session.sql(
    f"""
        CREATE OR REPLACE file format DEMO_DB.file_formats.csv_fileformat
            type = csv
            field_delimiter = ','
            skip_header = 1
            null_if = ('NULL','null')
            empty_field_as_null = TRUE;
    """
    )

result.show()

----------------------------------------------------
|"status"                                          |
----------------------------------------------------
|File format CSV_FILEFORMAT successfully created.  |
----------------------------------------------------



In [22]:
# create a file format object
result = session.sql(
    f"""
        CREATE OR REPLACE SCHEMA DEMO_DB.external_stages;
    """
    )

result.show()

------------------------------------------------
|"status"                                      |
------------------------------------------------
|Schema EXTERNAL_STAGES successfully created.  |
------------------------------------------------



In [43]:
# Create stage object with integration object & file format object


result = session.sql(
    f"""
        CREATE OR REPLACE stage demo_db.external_stages.s3_stg
            URL = {cl.secrets['s3_bucket_for_stg']}
            STORAGE_INTEGRATION = s3_int
            FILE_FORMAT = demo_db.file_formats.csv_fileformat;
    """
    )

result.show()



-------------------------------------------
|"status"                                 |
-------------------------------------------
|Stage area S3_STG successfully created.  |
-------------------------------------------



In [41]:
print(cl.secrets['s3_bucket_for_stg'])

s3://inzs3bucket/


In [35]:
# describing the stage
result = session.sql(
    f"""
        desc stage DEMO_DB.EXTERNAL_STAGES.S3_STG;
    """
    )

result.show()

-------------------------------------------------------------------------------------------------------------------------
|"parent_property"   |"property"           |"property_type"  |"property_value"                     |"property_default"  |
-------------------------------------------------------------------------------------------------------------------------
|STAGE_FILE_FORMAT   |FORMAT_NAME          |String           |demo_db.file_formats.csv_fileformat  |                    |
|STAGE_COPY_OPTIONS  |ON_ERROR             |String           |ABORT_STATEMENT                      |ABORT_STATEMENT     |
|STAGE_COPY_OPTIONS  |SIZE_LIMIT           |Long             |                                     |                    |
|STAGE_COPY_OPTIONS  |PURGE                |Boolean          |false                                |false               |
|STAGE_COPY_OPTIONS  |RETURN_FAILED_ONLY   |Boolean          |false                                |false               |
|STAGE_COPY_OPTIONS  |EN

In [45]:
# listing the files to know if we are able to access the stage
result = session.sql(
    f"""
        list @DEMO_DB.EXTERNAL_STAGES.S3_STG;
    """
    )

result.show()

-------------------------------------------------------------------------------------------------------------------------------
|"name"                                           |"size"  |"md5"                             |"last_modified"                |
-------------------------------------------------------------------------------------------------------------------------------
|s3://inzs3bucketlearn/sp_avro/userdata1.avro     |93561   |e60871d1105f230b90d5ae2f8d9481ce  |Wed, 21 Feb 2024 11:05:57 GMT  |
|s3://inzs3bucketlearn/sp_csv/employee01.csv      |342     |8e77d4649e70c82d47e0d4d8b76a13ee  |Wed, 21 Feb 2024 11:08:40 GMT  |
|s3://inzs3bucketlearn/sp_csv/employee02.csv      |391     |2ccf600775f9db9c78b49964cb5ff821  |Wed, 21 Feb 2024 11:08:41 GMT  |
|s3://inzs3bucketlearn/sp_csv/employee03.csv      |408     |a3f1099a25248926a1d9fa32947ca289  |Wed, 21 Feb 2024 11:08:41 GMT  |
|s3://inzs3bucketlearn/sp_csv/employee04.csv      |387     |9906f9f4449edb68f7570737d6999b3b  |Wed, 21 F